<a href="https://colab.research.google.com/github/anacasicande/Modelamiento/blob/main/tarea_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import sympy as sp
import numpy as np


P = 9.4573  # Presión en bar
T = 350.0   # Temperatura en K
R = 0.08314  # Constante de los gases en J/mol·K (convertir después a L·bar/mol·K)

# Propiedades críticas del n-butano
Tc = 425.2  # Temperatura crítica en K
Pc = 37.96  # Presión crítica en bar
tr = T / Tc  # Temperatura reducida
pr = P / Pc  # Presión reducida
Z = sp.Symbol('Z')  # Creo una variable simbolica para poder derivar usando la funcion diff

# Parámetros de la ecuación de Redlich-Kwong
q = (0.42748 / 0.08664) * (tr**(-3/2))  #  Formula de q que usa los valores
b = ((0.08664 * pr )/ tr)

# Función para Z_vapor usando Redlich-Kwong
def Rk_vapor(Z):  # Defino Rk_vapor como una funcion que recibe de parametros una Z_vapor
    return Z - (1 + b - q * b * ((Z - b) / (Z * (Z + b))))  # Retorna la ecuacion de Rk_vapor

# Derivada de la función Z_vapor
def dRk_vapor(Z_value):  # Defino dRk_vapor como una funcion que recibe de parametros una variable simbolica
    derivadaV = sp.diff(Rk_vapor(Z), Z)  # Derivo usando la funcion sp.diff que recibe como parametros la funcion a derivar y una variable simbolica
    return derivadaV.subs(Z, Z_value).evalf()  #  Retorna la derivada evaluada en el valor "Z"

# Función para Z_liquid usando Redlich-Kwong
def Rk_liquid(Z):  # Defino Rk_liquid como una funcion que recibe de parametros una Z_liquid
    return Z - (b + Z * (Z + b) * ((1 + b - Z) / (q * b)))  # Retorna la ecuacion de Rk_liquid

# Derivada de la función Z_liquid
def dRk_liquid(Z_value):  # Defino dRk_liquid como una funcion que recibe de parametros una variable simbolica
    derivadaL = sp.diff(Rk_liquid(Z), Z)  # Derivo usando la funcion sp.diff que recibe como parametros la funcion a derivar y una variable simbolica
    return derivadaL.subs(Z, Z_value).evalf()  #  Retorna la derivada evaluada en el valor "Z"

# Método de Newton-Raphson
import numpy as np

def newton_raphson(func, func_prime, Z, tol=1e-10, max_iter=200):
    x = Z
    error = 1
    it = 0

    while error > tol and it < max_iter:
        try:
            fx = func(x)
            fpx = func_prime(x)

            if abs(fpx) < tol:
                print("La derivada es muy pequeña, no se puede continuar")
                return None

            # Fórmula de Newton-Raphson
            x_new = x - fx / fpx
            error = np.abs(fx / fpx)
            print(f"Iteración {it}: x = {x_new}, error = {error}")

            # Verificar la convergencia
            if abs(x_new - x) < tol:
                print(f"Convergió en la iteración {it}")
                return x_new

            # Actualizar x para la siguiente iteración
            x = x_new
            it += 1

        except ZeroDivisionError:
            print("División por cero en la iteración", it)
            return None

    if it == max_iter:
        print("No converge después del número máximo de iteraciones")
        return False
    else:
        return x





# Estimaciones iniciales para los factores de compresibilidad Z
Z_guess_vapor = 1  # # El libro de Smith, Van Ness sugiere inicializar Z en 1 para vapor saturado.
Z_guess_liquid = 0.005  # Aproximación para el líquido

# Calcular Z usando Newton-Raphson
Z_vapor = newton_raphson(Rk_vapor, dRk_vapor, Z_guess_vapor)
Z_liquid = newton_raphson(Rk_liquid, dRk_liquid, Z_guess_liquid)

# Convertir Z a volúmenes molares Vm (en L/mol) y luego a cm^3/mol
Vm_vapor_cm3 = Z_vapor * R * T / P *1000 if Z_vapor else None
Vm_liquid_cm3 = Z_liquid * R * T / P * 1000  if Z_liquid else None

Z_vapor, Z_liquid, Vm_vapor_cm3, Vm_liquid_cm3



Iteración 0: x = 0.836335012891233, error = 0.163664987108767
Iteración 1: x = 0.830364789487029, error = 0.00597022340420460
Iteración 2: x = 0.830353561610826, error = 0.0000112278762026935
Iteración 3: x = 0.830353561570580, error = 4.02459783225915E-11
Convergió en la iteración 3
Iteración 0: x = 0.0331227783871731, error = 0.0281227783871731
Iteración 1: x = 0.0421987232815132, error = 0.00907594489434009
Iteración 2: x = 0.0432902378155151, error = 0.00109151453400194
Iteración 3: x = 0.0433061338757576, error = 0.0000158960602424895
Iteración 4: x = 0.0433061372400761, error = 3.36431842542168E-9
Iteración 5: x = 0.0433061372400762, error = 1.28765749305476E-16
Convergió en la iteración 5


(0.830353561570580, 0.0433061372400762, 2554.90026626440, 133.247891845345)